In [20]:
%matplotlib inline

In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [22]:
landings = pd.read_csv("Meteorite_Landings.csv")
thing = pd.read_csv("GLC03122015.csv")
fireball = pd.read_csv("Fireball_And_Bolide_Reports.csv")
comets = pd.read_csv("Near-Earth_Comets_-_Orbital_Elements.csv")

In [23]:
def analyze(dataset, cell, details = True):
    values = dataset[cell].unique()
    print(len(values))
    print(values)
    if(details):
        for value in values:
            print(value, ' -> ', len(dataset[dataset[cell]==value]))

In [24]:
# happily there is one data entry per metorite
# we can discard the name
print(len(landings["name"]))
print(len(landings["name"].unique()))

landings = landings.drop(["name", 'id'], axis = 1)
landings.head()

45716
45716


,nametype,recclass,mass (g),fall,year,reclat,reclong,GeoLocation
0,Valid,L5,21.0,Fell,01/01/1880 12:00:00 AM,50.77500,6.08333,"(50.775, 6.08333)"
1,Valid,H6,720.0,Fell,01/01/1951 12:00:00 AM,56.18333,10.23333,"(56.18333, 10.23333)"
2,Valid,EH4,107000.0,Fell,01/01/1952 12:00:00 AM,54.21667,-113.00000,"(54.21667, -113.0)"
3,Valid,Acapulcoite,1914.0,Fell,01/01/1976 12:00:00 AM,16.88333,-99.90000,"(16.88333, -99.9)"
4,Valid,L6,780.0,Fell,01/01/1902 12:00:00 AM,-33.16667,-64.95000,"(-33.16667, -64.95)"


In [25]:
#Useful
analyze(thing, "date_", False)

2148
['03/07/2011 08:00:00 AM +0000' '04/01/2008 07:00:00 AM +0000'
 '01/30/2015 08:00:00 AM +0000' ... '12/22/2009 08:00:00 AM +0000'
 '09/27/2007 07:00:00 AM +0000' '11/18/2007 08:00:00 AM +0000']


In [26]:
#Useless
analyze(thing, "time_")

3
['12/30/1899 08:00:00 AM +0000' nan '01/12/1900 08:00:00 AM +0000']
12/30/1899 08:00:00 AM +0000  ->  1268
nan  ->  0
01/12/1900 08:00:00 AM +0000  ->  1


In [27]:
analyze(thing, "hazard_typ")


2
['landslide' 'Lanslide']
landslide  ->  6787
Lanslide  ->  1


In [28]:
analyze(thing, "landslide_", False)
# TODO: Fix Landslide & landslide     rename - landslide_type

15
['Mudslide' 'Landslide' 'Complex' 'Other' 'Rock_Fall' 'Debris_Flow'
 'Rockfall' 'Snow_Avalanche' 'Unknown' 'Translational_Slide' 'mudslide'
 'Creep' 'Lahar' 'Riverbank_Collapse' 'landslide']


In [29]:
# too little to give a fuck
analyze(thing, "storm_name", False)

165
[nan 'Trami' 'Cyclone Giri' 'Hurricane Felix' '3 days of rain'
 'Cyclone Nargis' 'Hudhud' 'Typhoon Kompasu' 'and Rain'
 'Tropical Storm Richard' 'Wipha' 'Tropical Cyclone Hubert'
 'Typhoon Morakot' 'Hurricane Tomas' 'Manuel' 'Lusi' 'Kong Rey'
 'Typhoon Ketsana' 'Utor' 'Tropical Storm Olga' 'Typhoon Sinlaku'
 'Typhoon Fanapi' 'Storm Xynthia' 'Luis' 'Typhoon Sepat' 'Typhoon Megi'
 'Soulik' 'Hurricane Karl and Tropical Storm Matthew'
 'Supertyphoon Juan (Megi)' 'Typhoon Chedeng (Songda)' 'Rammasun'
 'Hurricane Beatriz' 'Santi' 'Tropical Storm Meranti'
 'Tropical Storm Tomas' 'Mangkhut' 'Tropical Storm Dodong (Sarika)'
 'Hurricane Dora' 'Cyclone Wilma' 'Tropical Storm Alma'
 'Tropical Depression Parma' 'Typhoon Ondoy (Ketsana)' 'Neoguri (Florita)'
 'Hurricane Frank' 'Haiyan' 'Tropical Cyclone Wilma' 'Toraji'
 'Tropical Cyclone Maring' 'Typhoon No. 2 and March 11th earthquake'
 'Usagi' 'Agaton' 'Hurricane Sandy' '1 hour of rain'
 'Tropical Storm Kammuri' 'Hurricane Dean' 'Typhoon Chanth

In [30]:
analyze(thing, "landslide1", True)
#TODO: filter & rename - severity

10
['Medium' 'Small' 'Large' 'Very_large' 'unknown' 'large' 'medium' 'small'
 'Extra Large' nan]
Medium  ->  4851
Small  ->  1277
Large  ->  482
Very_large  ->  79
unknown  ->  26
large  ->  23
medium  ->  33
small  ->  14
Extra Large  ->  1
nan  ->  0


In [31]:
#BS
analyze(thing, "cat_src", True)

3
['glc' 'test' nan]
glc  ->  6695
test  ->  92
nan  ->  0


In [32]:
#TODO: figure out wtf distance distance means

In [33]:
#TODO: near vs nearest_pl

In [34]:
thing = thing.drop(['the_geom', "hazard_typ", 'OBJECTID', 'id', 'time_', "storm_name", 'source_nam', 'fatalities', 'injuries', 'source_lin', 'location_a', 'photos_lin', 'cat_src', 'cat_id', 'countrynam', 'adminname1', 'adminname2', 'population', 'countrycod', 'continentc', 'key_', 'version', 'user_id', 'tstamp', 'changeset_', 'distance', 'near', 'nearest_pl'], axis = 1)

In [37]:
thing.rename(columns={'landslide1':'severity', 'landslide_':'landslide_type', 'date_':'date'}, inplace=True)

SyntaxError: invalid syntax (<ipython-input-37-908017fc249c>, line 1)

In [ ]:
thing.head()